In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import math
from datetime import datetime, timedelta
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from scipy.stats import entropy


# Définir les scénarios et le chemin d'accès
training_scenarios = [9]
base_path = "/content/drive/MyDrive/Training scenarios/"

training_files = []

for scenario_id in training_scenarios:
    file_name = f"{base_path}scenario{scenario_id}.binetflow"
    training_files.append(pd.read_csv(file_name, delimiter=",", low_memory=False))



# ==========================
# 1. Prétraitement des données
# ==========================
def preprocess_flags(df, is_test=False):
    """
    Convertit le champ 'StartTime' en datetime, filtre les flux
    avant les 25 premières minutes, convertit les labels en 0/1 et
    conserve uniquement les flux TCP. Renomme 'State' en 'Flags' si nécessaire.
    """
    df['StartTime'] = pd.to_datetime(df['StartTime'])
    min_time = df['StartTime'].min()
    threshold_time = min_time + pd.Timedelta(minutes=25)
    df = df[df['StartTime'] >= threshold_time].copy()
    print("Après filtrage temporel, forme du DataFrame :", df.shape)

    # Convertir le label : 1 si "botnet" est présent dans le label, sinon 0
    df['Label'] = df['Label'].apply(lambda x: 1 if 'botnet' in str(x).lower() else 0)

    # Normaliser et filtrer le protocole pour ne garder que TCP
    df['Proto'] = df['Proto'].str.strip().str.lower()
    df = df[df['Proto'] == 'tcp'].copy()

    # Renommer la colonne 'State' en 'Flags' si nécessaire
    if 'State' in df.columns and 'Flags' not in df.columns:
        df.rename(columns={'State': 'Flags'}, inplace=True)

    return df

def get_rare_combos(df, rare_threshold=0.01):
    """
    Retourne une liste des combinaisons de flags rares (qui apparaissent en dessous d'un seuil).
    """
    combo_counts = df['Flags'].value_counts(normalize=True)
    rare_combos = combo_counts[combo_counts < rare_threshold].index.tolist()
    return rare_combos

# ==========================
# 2. Extraction de caractéristiques améliorée (avec entropie temporelle)
# ==========================
def compute_temporal_flag_entropy(df, window_size=10, step=1):
    """
    Calcule l'entropie moyenne temporelle des combinaisons de flags TCP pour chaque adresse IP.

    Paramètres :
        df (DataFrame) : Données prétraitées contenant les colonnes 'SrcAddr', 'StartTime', et 'Flags'.
        window_size (int) : Taille de la fenêtre glissante utilisée pour calculer l'entropie.
        step (int) : Décalage entre chaque fenêtre glissante.

    Retourne :
        Series : Entropie moyenne pour chaque IP, indiquant la variabilité temporelle des flags.
    """

    # Initialiser un dictionnaire pour stocker les entropies moyennes par IP
    ip_entropy = {}

    # Regrouper les flux par adresse IP source
    for ip, group in df.groupby('SrcAddr'):

        # Trier les flux par ordre chronologique pour capturer l'évolution temporelle
        group_sorted = group.sort_values('StartTime')

        # Initialiser la liste qui stockera les entropies pour chaque fenêtre glissante
        entropies = []

        # Extraire la séquence chronologique des flags TCP
        flags_list = group_sorted['Flags'].tolist()

        # Parcourir la liste des flags avec une fenêtre glissante
        for i in range(0, len(flags_list) - window_size + 1, step):

            # Sélectionner une fenêtre de flags consécutifs
            window = flags_list[i:i + window_size]

            # Calculer la fréquence de chaque combinaison de flags dans la fenêtre
            freq = {}
            for f in window:
                freq[f] = freq.get(f, 0) + 1

            # Normaliser les fréquences pour obtenir une distribution de probabilités
            total = sum(freq.values())
            norm_freq = [count / total for count in freq.values()]

            # Calculer l'entropie (base 2) de cette distribution de probabilités
            ent = entropy(norm_freq, base=2)

            # Ajouter l'entropie calculée à la liste des entropies de cette IP
            entropies.append(ent)

        # Stocker l'entropie moyenne calculée pour cette IP
        ip_entropy[ip] = np.mean(entropies) if entropies else 0

    # Retourner les résultats sous forme de série Pandas
    return pd.Series(ip_entropy, name='temporal_flag_entropy')


def extract_features_enhanced(df, rare_combos):
    """
    Extrait un ensemble de caractéristiques enrichies pour chaque IP.
      - Indicateurs binaires pour les flags (A, S, F, R, P)
      - Ratio SYN/FIN
      - Nombre de flux par IP
      - Durée moyenne des flux
      - Entropie globale des combinaisons de flags
      - Entropie temporelle (basée sur des fenêtres glissantes)
      - Histogramme normalisé des combinaisons de flags
    """
    # Création des indicateurs binaires pour chaque flag
    flags = ['A', 'S', 'F', 'R', 'P']
    for flag in flags:
        df[f'has_{flag}'] = df['Flags'].apply(lambda x: 1 if flag in x else 0)

    # Agrégation par IP : moyenne des indicateurs binaires
    individual_features = df.groupby('SrcAddr')[['has_A', 'has_S', 'has_F', 'has_R', 'has_P']].mean()

    # Calcul du ratio SYN/FIN
    syn_count = df.groupby('SrcAddr')['has_S'].sum()
    fin_count = df.groupby('SrcAddr')['has_F'].sum()
    syn_fin_ratio = (syn_count / (fin_count + 1e-9)).rename('syn_fin_ratio')

    # Nombre de flux par IP
    flow_count = df.groupby('SrcAddr').size().rename('flow_count')

    # Durée moyenne des flux par IP
    avg_duration = df.groupby('SrcAddr')['Dur'].mean().rename('avg_duration')

    # Construction de l'histogramme des combinaisons de flags
    df['FlagCombo'] = df['Flags'].apply(lambda x: 'Other' if x in rare_combos else x)
    combo_hist = df.groupby(['SrcAddr', 'FlagCombo']).size().unstack(fill_value=0)
    total_flows = combo_hist.sum(axis=1)
    norm_combo_hist = combo_hist.div(total_flows, axis=0)

    # Entropie globale basée sur l'histogramme normalisé
    global_entropy = norm_combo_hist.apply(lambda row: entropy(row, base=2), axis=1).rename('flag_entropy')

    # Calcul de l'entropie temporelle à l'aide de fenêtres glissantes
    temporal_entropy = compute_temporal_flag_entropy(df, window_size=10, step=1)

    # Combinaison de toutes les caractéristiques
    features = pd.concat([individual_features, syn_fin_ratio, flow_count, avg_duration, global_entropy, temporal_entropy], axis=1)
    features = pd.concat([features, norm_combo_hist], axis=1).fillna(0)

    return features


In [ ]:
# ==========================
# 3. Entraînement du modèle hybride (PCA + Classification)
# ==========================
def train_hybrid_model(train_dfs, n_components=10):
    """
    Prépare les données, extrait les caractéristiques enrichies, puis :
      - Agrège les labels par IP (1 si au moins un flux est malveillant)
      - Standardise les caractéristiques
      - Applique le PCA pour réduire la dimensionnalité
      - Entraîne une régression logistique sur les données projetées
    Retourne le PCA, la liste des combinaisons rares, le scaler, le classifieur,
    les caractéristiques standardisées et les labels agrégés par IP.
    """
    full_train = pd.concat([preprocess_flags(df) for df in train_dfs])
    rare_combos = get_rare_combos(full_train)
    features = extract_features_enhanced(full_train, rare_combos)

    # Agréger les labels par IP (au moins un flux malveillant => label 1)
    ip_labels = full_train.groupby('SrcAddr')['Label'].max()
    features = features.loc[ip_labels.index]

    # Standardisation des caractéristiques
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)
    features_scaled = pd.DataFrame(features_scaled, index=features.index, columns=features.columns)

    # Application du PCA sur les caractéristiques standardisées
    pca = PCA(n_components=n_components, svd_solver='full')
    pca.fit(features_scaled)
    features_pca = pca.transform(features_scaled)

    # Entraînement du classifieur (régression logistique) sur l'espace PCA
    clf = LogisticRegression(class_weight='balanced', max_iter=1000)
    clf.fit(features_pca, ip_labels.loc[features.index])

    # Calcul de l'erreur de reconstruction pour référence
    features_reconstructed = pca.inverse_transform(features_pca)
    reconstruction_errors = np.linalg.norm(features_scaled - features_reconstructed, axis=1)
    print("Erreur de reconstruction moyenne sur l'entraînement :", np.mean(reconstruction_errors))

    return pca, rare_combos, scaler, clf, features_scaled, ip_labels


In [ ]:
# ==========================
# 4. Détection via la méthode hybride
# ==========================
def detect_anomalies_hybrid(test_df, pca, rare_combos, scaler, clf, train_feature_columns, norm_type='l2'):
    """
    Traite les données de test : prétraitement, extraction et standardisation des caractéristiques,
    projection dans l'espace PCA et utilisation du classifieur pour prédire la probabilité d'être malveillant.
    Retourne les probabilités d'anomalie, le DataFrame prétraité et l'erreur de reconstruction.
    """
    preprocessed_test = preprocess_flags(test_df, is_test=True)
    test_features = extract_features_enhanced(preprocessed_test, rare_combos)
    test_features = test_features.reindex(columns=train_feature_columns, fill_value=0)

    # Appliquer le scaler sur les données de test
    test_features_scaled = scaler.transform(test_features)
    test_features_scaled = pd.DataFrame(test_features_scaled, index=test_features.index, columns=test_features.columns)

    # Projection dans l'espace PCA
    test_features_pca = pca.transform(test_features_scaled)

    # Prédiction de la probabilité d'être malveillant via la régression logistique
    anomaly_prob = clf.predict_proba(test_features_pca)[:, 1]
    anomaly_prob = pd.Series(anomaly_prob, index=test_features.index)

    # Calcul optionnel de l'erreur de reconstruction (norme L2 ou L1)
    if norm_type == 'l2':
        recon_error = np.linalg.norm(test_features_scaled - pca.inverse_transform(test_features_pca), axis=1)
    elif norm_type == 'l1':
        recon_error = np.sum(np.abs(test_features_scaled - pca.inverse_transform(test_features_pca)), axis=1)
    else:
        raise ValueError("norm_type must be either 'l2' or 'l1'")
    recon_error = pd.Series(recon_error, index=test_features.index)

    return anomaly_prob, preprocessed_test, recon_error


In [ ]:
# ==========================
# 5. Filtrage supplémentaire par nombre minimum de flux par IP
# ==========================
def filter_by_flow_count(preprocessed_test, min_flows=50):
    """
    Retourne la liste des IP ayant au moins 'min_flows' flux.
    """
    flow_counts = preprocessed_test.groupby('SrcAddr').size()
    valid_ips = flow_counts[flow_counts >= min_flows].index
    return valid_ips

In [ ]:
# ==========================
# 6. Fonction d'évaluation
# ==========================
def evaluate_performance(scores, preprocessed_test, threshold):
    """
    Évalue la performance en agrégant les labels par IP (vrai label = max(Label)) et en comparant
    aux prédictions basées sur le seuil.
    Retourne les métriques et la matrice de confusion.
    """
    y_true = preprocessed_test.groupby('SrcAddr')['Label'].max()
    y_pred = (scores > threshold).astype(int)
    y_true, y_pred = y_true.align(y_pred, join='right', fill_value=0)

    TP = np.sum((y_pred == 1) & (y_true == 1))
    FP = np.sum((y_pred == 1) & (y_true == 0))
    TN = np.sum((y_pred == 0) & (y_true == 0))
    FN = np.sum((y_pred == 0) & (y_true == 1))

    epsilon = 1e-9
    metrics = {
        'FPR': FP / (TN + FP + epsilon),
        'TPR': TP / (TP + FN + epsilon),
        'TNR': TN / (TN + FP + epsilon),
        'FNR': FN / (TP + FN + epsilon),
        'Precision': TP / (TP + FP + epsilon),
        'Accuracy': (TP + TN) / (TP + TN + FP + FN + epsilon),
        'ErrorRate': (FP + FN) / (TP + TN + FP + FN + epsilon),
        'F1-Score': 2 * TP / (2 * TP + FP + FN + epsilon)
    }
    return metrics, (TP, FP, TN, FN)

In [ ]:
# ==========================
# 7. Fonction d'optimisation du seuil
# ==========================
def optimize_threshold(scores, preprocessed_test, percentiles=np.arange(10, 100, 1)):
    """
    Parcourt une plage de percentiles pour fixer un seuil qui maximise le F1-score.
    Retourne le meilleur seuil, les métriques associées et la matrice de confusion.
    """
    best_threshold = None
    best_f1 = -1
    best_metrics = None
    best_conf = None

    for perc in percentiles:
        threshold = np.percentile(scores, perc)
        metrics, conf = evaluate_performance(scores, preprocessed_test, threshold)
        if metrics['F1-Score'] > best_f1:
            best_f1 = metrics['F1-Score']
            best_threshold = threshold
            best_metrics = metrics
            best_conf = conf
        print(f"Percentile: {perc}, Seuil: {threshold:.4f}, F1-Score: {metrics['F1-Score']:.4f}")

    print("\nSeuil optimal:", best_threshold)
    return best_threshold, best_metrics, best_conf

In [ ]:
# Load test file
scenario_test_id = 9
test_df = pd.read_csv(f"{base_path}scenario{scenario_test_id}.binetflow", delimiter=",", low_memory=False)

# ---- Phase d'entraînement ----
pca, rare_combos, scaler, clf, train_features_scaled, ip_labels = train_hybrid_model(training_files, n_components=10)
print("Modèle hybride entraîné. Forme des features d'entraînement :", train_features_scaled.shape)

# ---- Phase de détection ----
anomaly_prob, preprocessed_test, recon_error = detect_anomalies_hybrid(test_df, pca, rare_combos, scaler, clf, train_features_scaled.columns, norm_type='l2')
print("Probabilités d'anomalie (aperçu) :\n", anomaly_prob.head())

# ---- Filtrage supplémentaire par nombre de flux ----
valid_ips = filter_by_flow_count(preprocessed_test, min_flows=50)
filtered_scores = anomaly_prob[anomaly_prob.index.isin(valid_ips)]
filtered_preprocessed_test = preprocessed_test[preprocessed_test['SrcAddr'].isin(valid_ips)]

# ---- Optimisation du seuil ----
best_threshold, best_metrics, best_conf_matrix = optimize_threshold(filtered_scores, filtered_preprocessed_test, percentiles=np.arange(10, 100, 1))

print("\n=== Matrice de confusion optimale ===")
print(f"TP: {best_conf_matrix[0]}, FP: {best_conf_matrix[1]}")
print(f"FN: {best_conf_matrix[2]}, TN: {best_conf_matrix[3]}\n")
print("=== Métriques de performance optimales ===")
metrics_df = pd.DataFrame([best_metrics], index=['Hybrid FLAGS PCA'])
print(metrics_df.round(4).T.to_markdown())



Après filtrage temporel, forme du DataFrame : (2087505, 15)
Erreur de reconstruction moyenne sur l'entraînement : 1.3186394607139016
Modèle hybride entraîné. Forme des features d'entraînement : (16938, 20)
Après filtrage temporel, forme du DataFrame : (2087505, 15)
Probabilités d'anomalie (aperçu) :
 1.144.169.55     1.896796e-10
1.144.233.103    3.625273e-11
1.144.62.178     3.980249e-13
1.148.65.189     3.486458e-12
1.152.83.190     7.412210e-13
dtype: float64
Percentile: 10, Seuil: 0.0000, F1-Score: 0.0775
Percentile: 11, Seuil: 0.0000, F1-Score: 0.0784
Percentile: 12, Seuil: 0.0000, F1-Score: 0.0794
Percentile: 13, Seuil: 0.0000, F1-Score: 0.0800
Percentile: 14, Seuil: 0.0000, F1-Score: 0.0810
Percentile: 15, Seuil: 0.0000, F1-Score: 0.0820
Percentile: 16, Seuil: 0.0000, F1-Score: 0.0830
Percentile: 17, Seuil: 0.0000, F1-Score: 0.0837
Percentile: 18, Seuil: 0.0000, F1-Score: 0.0847
Percentile: 19, Seuil: 0.0000, F1-Score: 0.0858
Percentile: 20, Seuil: 0.0000, F1-Score: 0.0870
Perce